In [1]:
from pypylon import pylon
import time
import cv2
import queue

# connecting to the first available camera
camera = pylon.InstantCamera(pylon.TlFactory.GetInstance().CreateFirstDevice())
camera.Open()


# if available choose a format that is
# computationally inexpensive to convert to your desired output format
camera.PixelFormat = "BGR8"
# or the YUV formats

class CaptureConvert(pylon.ImageEventHandler):
    def __init__(self):
        super().__init__()
        # converting to opencv bgr format
        self.converter = pylon.ImageFormatConverter()
        self.converter.OutputPixelFormat = pylon.PixelType_BGR8packed
        self.converter.OutputBitAlignment = pylon.OutputBitAlignment_MsbAligned

        self.video_queue = queue.Queue(maxsize=2)

    def OnImageGrabbed(self, cam, grab_result):
        if grab_result.GrabSucceeded():
            image = self.converter.Convert(grab_result)
            try:
                self.video_queue.put_nowait(image.GetArray())
            except queue.Full:
                # if queue depth > 2 your display thread is too slow
                # try limiting the camera framerate or use faster display framework
                print("consumer thread too slow frame:", grab_result.FrameNumber)


# register the background handler
capture = CaptureConvert()

camera.RegisterImageEventHandler(capture, pylon.RegistrationMode_ReplaceAll, pylon.Cleanup_None)

# start grabbing using background pylon thread
camera.StartGrabbing(pylon.GrabStrategy_LatestImageOnly, pylon.GrabLoop_ProvidedByInstantCamera)

start_time = time.time()
counter = 0
fps_real = camera.ResultingFrameRate.Value
fps_measured = 0

while camera.IsGrabbing():
    # Access the image data
    img = capture.video_queue.get(timeout=5000)

    # fps display measured over 1s run
    counter += 1
    if (time.time() - start_time) > 1:
        elapsed = time.time() - start_time
        fps_measured = int(counter / elapsed)
        counter = 0
        start_time = time.time()

    cv2.putText(img, "{} FPS of {} FPS camera".format(fps_measured, fps_real), (200, 20),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

    # image display
    cv2.imshow('title', img)

    # keep gui running
    k = cv2.waitKey(1)
    if k == 27:
        break

# Releasing the resource
camera.StopGrabbing()
camera.Close()

cv2.destroyAllWindows()

ModuleNotFoundError: No module named 'pypylon'